In [2]:
import os
import json
from pathlib import Path
import gzip
import hashlib
import shutil

import pandas as pd
import pygeohash
import s3fs

import uuid
import math

In [3]:
endpoint_url = 'https://storage.budsc.midwest-datascience.com'

In [4]:
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')

In [5]:
if results_dir.exists():
    shutil.rmtree(results_dir)
results_dir.mkdir(parents=True, exist_ok=True)

In [6]:
def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]
            
    return records

In [7]:
def flatten_record(record):
    flat_record = dict()
    for key, value in record.items():
        if key in ['airline', 'src_airport', 'dst_airport']:
            if isinstance(value, dict):
                for child_key, child_value in value.items():
                    flat_key = '{}_{}'.format(key, child_key)
                    flat_record[flat_key] = child_value
        else:
            flat_record[key] = value
            
    return flat_record

In [8]:
def create_flatten_dataset():
    records = read_jsonl_data()
    return pd.DataFrame.from_records([flatten_record(record) for record in records])

In [9]:
df = create_flatten_dataset()
df['key'] = df['src_airport_iata'].astype(str) + df['dst_airport_iata'].astype(str) + df['airline_iata'].astype(str)

In [10]:
df['key']

0        AERKZN2B
1        ASFKZN2B
2        ASFMRV2B
3        CEKKZN2B
4        CEKOVB2B
           ...   
67658    WYAADLZL
67659    DMEFRUZM
67660    FRUDMEZM
67661    FRUOSSZM
67662    OSSFRUZM
Name: key, Length: 67663, dtype: object

In [32]:
df['kv_key'] = df['key'].astype(str).str[0]
df[['key','kv_key']]

,key,kv_key
0,AERKZN2B,A
1,ASFKZN2B,A
2,ASFMRV2B,A
3,CEKKZN2B,C
4,CEKOVB2B,C
...,...,...
67658,WYAADLZL,W
67659,DMEFRUZM,D
67660,FRUDMEZM,F
67661,FRUOSSZM,F


In [12]:
parquet_path = results_dir.joinpath('routes-flattened.parquet')
print(parquet_path)

/home/jovyan/dsc650/dsc650/assignments/assignment07/results/routes-flattened.parquet


In [17]:
import pyarrow as pa
import pyarrow.parquet as pq 

In [22]:
table = pa.Table.from_pandas(df)
kv_dir = results_dir.joinpath('kv')
kv_dir.mkdir(parents=True, exist_ok=True)
# pq.write_table(table,f'{parquet_path}')

In [19]:
partitions = (
    ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
    ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
    ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
    ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
)

In [30]:
new_partition = []
for letter in partitions:
    if letter[0] == letter[1]:
        # print(letter[0])
        new_partition.append(letter[0])
    else:
        # print(f'{letter[0]}-{letter[1]}')
        new_partition.append(f'{letter[0]}-{letter[1]}')

In [39]:
def kv_key(first_letter):
    for letter in partitions:
        if first_letter in letter:
            if letter[0] == letter[1]:
                return (print(letter[0]))
            else:
                return (print(f'{letter[0]}-{letter[1]}'))

In [43]:
kv_key('P')

O-P


In [52]:
df['kv_key'].isnull().any()

False

In [57]:
df['kv_key']

0        A
1        A
2        A
3        C
4        C
        ..
67658    W
67659    D
67660    F
67661    F
67662    O
Name: kv_key, Length: 67663, dtype: object

In [62]:
df['kv_key_new'] = df.apply(lambda x: kv_key(x['kv_key']), axis=1)
concat(df,df['kv_key_new'])

A
A
A
C-D
C-D
C-D
C-D
C-D
C-D
E-F
E-F
G-H
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
M
N
N
N
N
N
N
N
O-P
O-P
S-T
S-T
S-T
S-T
S-T
U
U
U
B
B
C-D
G-H
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
M
O-P
O-P
O-P
O-P
U
U
Y-Z
A
C-D
C-D
G-H
I-J
I-J
K-L
K-L
K-L
K-L
K-L
O-P
O-P
O-P
O-P
S-T
S-T
S-T
A
A
A
B
B
B
B
B
C-D
C-D
C-D
C-D
K-L
N
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
B
B
C-D
G-H
G-H
G-H
G-H
O-P
S-T
U
U
U
B
B
B
G-H
K-L
K-L
Q-R
S-T
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
A
A
A
A
A
A
A
A
A
A
A
A
A
A
G-H
G-H
G-H
G-H
G-H
G-H
I-J
I-J
K-L
K-L
K-L
K-L
K-L
K-L
M
M
O-P
O-P
O-P
O-P
S-T
S-T
V
V
V
V
V
A
A
A
A
A
K-L
B
B
C-D
C-D
C-D
C-D
C-D
G-H
K-L
K-L
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
O-P
O-P
Q-R
S-T
S-T
Y-Z
A
B
B
B
B
B
C-D
C-D
C-D
C-D
C-D
C-D
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
O-P
O-P
O-P
O-P
O-P
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
S-T
S-T
S-T
U
U
U
U
U
U
V
V
B
B
C-D
C-D
I-J
O-P
O-P
S-T
S-T
S-T
K-L
K-L
K-L
K-L
M
A
A
A
B
B
B
B
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
E-F
E-F
E-F
E-F
K-L
K-L
K-L
K-L
K-L
O-P
O-P
Q-R
Q-R
Q-R
A
A
W-X
Y-Z
Y-Z
Y-

Q-R
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
A
A
A
A
A
B
C-D
C-D
C-D
E-F
E-F
E-F
G-H
G-H
G-H
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
K-L
K-L
O-P
S-T
S-T
A
A
A
A
B
B
B
B
C-D
C-D
C-D
C-D
C-D
E-F
E-F
M
M
M
M
N
N
O-P
O-P
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
S-T
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
A
A
A
B
B
B
B
B
B
B
B
B
C-D
C-D
C-D
C-D
C-D
E-F
G-H
I-J
I-J
I-J
I-J
I-J
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
A
A
A
A
A
A
B
B
B
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
G-H
G-H
G-H
G-H
G-H
G-H
G-H
K-L
K-L
N
Q-R
Q-R
V
V
B
B
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
I-J
I-J
I-J
I-J
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
M
M
M
M
N
N
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
Q-R
S-T
S-T
S-T
S-T


O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P


G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
G-H
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
K-L
K-L
I-J
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
N
N
N
N
N
O-P
O-P
O-P
O-P
O-P
O-P
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
U
U
V
V
V
V
V
V
V
V
V
Y-Z
Y-Z
A
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
B
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
G-H
G-H
G-H
G-H
G-H
G-H
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-

G-H
G-H
G-H
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
N
N
N
N
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
Q-R
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
U
C-D
C-D
C-D
C-D
E-F
E-F
G-H
G-H
G-H
K-L
K-L


...

Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
A
A
A
A
A
B
B
B
B
B
C-D
C-D
C-D
C-D
C-D
G-H
M
M
M
M
M
M
M
M
M
M
M
M
O-P
O-P
O-P
O-P
O-P
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
A
A
A
B
B
B
B
B
B
B
B
B
B
B
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
E-F
E-F
E-F
E-F
G-H
G-H
G-H
G-H
G-H
I-J
I-J
K-L
K-L
K-L
K-L
K-L
K-L
K-L
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
N
N
N
N
N
N
N
N
N
N
N
N
O-P
O-P
O-P
O-P
O-P
O-P
O-P
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
S-T
V
V
Y-Z
Y-Z
Y-Z
B
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
G-H
S-T
S-T
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
M
N
N
O-P
S-T
S-T
S-T
S-T
W-X
W-X
W-X
W-X
W-X
Y-Z
B
C-

I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L


M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
N
N
N
N
N
N
N
N
N
N
N
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-P
O-

Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
Y-Z
B
B
C-D
C-D
C-D
C-D
E-F
E-F
G-H
G-H
G-H
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
I-J
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
N
S-T
S-T
S-T
V
V
V
A
C-D
C-D
K-L
K-L
K-L
K-L
K-L
K-L
S-T
A
A
B
B
C-D
C-D
C-D
C-D
C-D
C-D
C-D
E-F
E-F
G-H
I-J
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
N
O-P
O-P
Q-R
A
A
A
B
B
B
B
B
B
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
E-F
I-J
Q-R
I-J
I-J
I-J
I-J
K-L
K-L
K-L
K-L
K-L
K-L
K-L
K-L
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
Q-R
S-T
S-T
S-T
S-T
S-T
Y-Z
Y-Z
A
A
A
A
A
A
A
A
A
A
A
A
B
B
B
B
B
B
B
B
B
B
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
C-D
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F
E-F


NameError: name 'concat' is not defined

In [ ]:
df['kv_key_new'] = df['kv_key'].apply(kv_key(['kv_key']), axis=1)

In [63]:
df

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_timezone,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key,kv_key_new
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZN2B,A,None
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZN2B,A,None
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRV2B,A,None
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZN2B,C,None
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,7.0,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVB2B,C,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67658,4178,Regional Express,Qantas Airways,ZL,RXA,REX,Australia,True,6334.0,Whyalla Airport,...,9.5,O,Australia/Adelaide,airport,OurAirports,False,[SF3],WYAADLZL,W,None
67659,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,4029.0,Domodedovo International Airport,...,6.0,U,Asia/Bishkek,airport,OurAirports,False,[734],DMEFRUZM,D,None
67660,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,2912.0,Manas International Airport,...,3.0,N,Europe/Moscow,airport,OurAirports,False,[734],FRUDMEZM,F,None
67661,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,2912.0,Manas International Airport,...,6.0,U,Asia/Bishkek,airport,OurAirports,False,[734],FRUOSSZM,F,None


In [31]:
new_partition

['A',
 'B',
 'C-D',
 'E-F',
 'G-H',
 'I-J',
 'K-L',
 'M',
 'N',
 'O-P',
 'Q-R',
 'S-T',
 'U',
 'V',
 'W-X',
 'Y-Z']

In [ ]:
df['kv_key_new'] = df['key'].astype(str).str[0]
df[['key','kv_key']]

In [21]:
pq.write_to_dataset(
    table,
    root_path=f'{kv_dir}',
    partition_cols=['kv_key']
)